In [4]:
import ollama

ModuleNotFoundError: No module named 'ollama'

In [ ]:
messages = [{"role" : "system", "content": "You will get a message from a user, you will take that message and deduct the general topic, limit the response to a maximum of five words"},
             {"role" : "user", "content": "Today I feel like asking questions about Recurrent neural networks and their effect within the text analysis subject"}]

In [ ]:
response = ollama.chat(model="llama3:8b", messages=messages)

In [ ]:
topic = response["message"]["content"]

In [ ]:
type(topic)

str

In [ ]:
print(response['message']['content'])

Text Analysis RNN Effect


In [ ]:
interviewing = [
    {
        "role": "system",
        "content": f"""
You are Mr. Ollama, an interviewer. 
You will ask one multiple-choice question (QCM) with a single correct answer.
The topic is: {topic}.

Your entire output must be **valid JSON**, with **no text or formatting before or after**.
Use this exact structure:

{{
  "question": "Your question here, including options A), B), C), D).",
  "answer": "A"  // The correct option letter, always as a string
}}

Rules:
- The JSON must be strictly valid according to RFC 8259.
- Always enclose strings in double quotes.
- Never include code fences, markdown, explanations, or comments.
- Only output one JSON object.
"""
    }
]


In [ ]:
QuestionsAskedByOllama = ollama.chat(messages=interviewing, model="llama3:8b")

In [ ]:
print(QuestionsAskedByOllama["message"]["content"])

{
  "question": "Which effect is often observed when using Recurrent Neural Networks (RNNs) for text analysis?",
  "answer": "A"
}


In [ ]:
print(QuestionsAskedByOllama["message"])

role='assistant' content='{\n  "question": "Which effect is often observed when using Recurrent Neural Networks (RNNs) for text analysis?",\n  "answer": "A"\n}' thinking=None images=None tool_name=None tool_calls=None


In [ ]:
import json

In [ ]:
print(QuestionsAskedByOllama["message"]["content"])


{
  "question": "Which effect is often observed when using Recurrent Neural Networks (RNNs) for text analysis?",
  "answer": "A"
}


In [ ]:
parsed = json.loads(QuestionsAskedByOllama["message"]["content"])

In [ ]:
parsed["answer"]

'A'

In [ ]:
interviewingAnswer = [
    {
        "role": "system",
        "content": """
You are Mr. Ollama, an interviewer continuing a multiple-choice test (QCM).

You will receive two inputs:
1. A JSON object containing the original question and the correct answer.
2. A user's sentence or response (which may include or imply a choice like "I think it's A", "The answer is Overfitting", etc.).

Your job is to:
- Identify which choice (A, B, C, or D) the user selected or implied.
- Compare it with the correct answer.
- Respond as the interviewer, saying whether the user is right or wrong, and briefly explain the correct answer.

Your output must be **valid JSON only**, with the following exact structure:

{
  "detected_answer": "A",  // The choice letter you think the user meant
  "correct_answer": "A",   // The correct choice from the question
  "is_correct": true,      // true or false
  "interviewer_response": "Correct! Overfitting is indeed a common challenge in text analysis neural networks."
}

Rules:
- Always use double quotes for strings.
- Never output anything outside the JSON.
- If you cannot determine the user’s answer, set "detected_answer" to "unknown" and "is_correct" to false.
"""
    }
]


In [ ]:
question_data = {
    "question": "What is a common challenge when training a text analysis neural network? A) Overfitting B) Underfitting C) Class imbalance D) Data scarcity.",
    "answer": "A"
}

user_sentence = "I think it's underfitting."

response = ollama.chat(
    model="llama3:8b",
    messages=[
        *interviewingAnswer,
        {"role": "user", "content": json.dumps({
            "question_data": question_data,
            "user_response": user_sentence
        })}
    ]
)

print(response["message"]["content"])


{
  "detected_answer": "B",
  "correct_answer": "A",
  "is_correct": false,
  "interviewer_response": "That's not correct. Overfitting is a common challenge when training a text analysis neural network, as the model may learn to fit the noise in the training data rather than the underlying patterns."
}


In [ ]:
import json
import ollama
import re
from gtts import gTTS
from io import BytesIO
import pygame
import time
import threading
import asyncio
import concurrent.futures

pygame.mixer.init()

executor = concurrent.futures.ThreadPoolExecutor(max_workers=3)

def text_to_speech(text):
    """Convert text to speech and play it - non-blocking version"""
    def _tts_thread(text):
        try:
            mp3_fp = BytesIO()
            tts = gTTS(text=text, lang='en')
            tts.write_to_fp(mp3_fp)
            mp3_fp.seek(0)
            
            pygame.mixer.music.load(mp3_fp, 'mp3')
            pygame.mixer.music.play()
            
            while pygame.mixer.music.get_busy():
                time.sleep(0.1)
                
        except Exception as e:
            print(f"⚠️ Audio error: {e}")
    
    threading.Thread(target=_tts_thread, args=(text,), daemon=True).start()

def generate_questions_optimized(topic, num_questions=5):
    """Generate questions with optimized prompt and settings"""
    
    prompt = f"""
Generate exactly {num_questions} multiple-choice questions about {topic}.
Return ONLY valid JSON in this exact format:

[
  {{
    "question": "Clear question here?",
    "options": {{
      "A": "Option A",
      "B": "Option B", 
      "C": "Option C",
      "D": "Option D"
    }},
    "answer": "A"
  }}
]

Requirements:
- Only JSON output, no other text
- {num_questions} questions total
- Each question must be concise (max 15 words)
- Options should be short (max 8 words each)
- Ensure answers are factually correct
- Cover different aspects of {topic}
"""

    try:
        
        response = ollama.chat(
            model="llama3:8b",  
            messages=[{"role": "user", "content": prompt}],
            options={
                "temperature": 0.3,  
                "num_predict": 512,  
            }
        )
        
        content = response["message"]["content"].strip()
        content = re.sub(r'```json\s*|\s*```', '', content)
        
        questions = json.loads(content)
        
        for q in questions:
            if not all(k in q for k in ['question', 'options', 'answer']):
                raise ValueError("Invalid question structure")
                
        return questions[:num_questions]  
    except Exception as e:
        print(f"❌ Question generation failed: {e}")
        return generate_fallback_questions(topic, num_questions)

def generate_fallback_questions(topic, num_questions):
    """Fallback questions if generation fails"""
    print("🔄 Using fallback questions...")
    return [
        {
            "question": f"What is the main concept of {topic}?",
            "options": {
                "A": "Fundamental principle",
                "B": "Advanced technique", 
                "C": "Basic terminology",
                "D": "Historical context"
            },
            "answer": "A"
        }
    ] * min(num_questions, 3)

def evaluate_answer_optimized(question_data, user_answer):
    """Fast answer evaluation without LLM call"""
    
    correct_answer = question_data.get('answer', '').upper()
    user_answer = user_answer.upper()
    
    is_correct = user_answer == correct_answer
    
    if is_correct:
        feedback = "Correct! Well done."
    else:
        correct_option = question_data['options'].get(correct_answer, 'the correct option')
        feedback = f"Incorrect. The right answer was {correct_answer}: {correct_option}"
    
    return {
        "detected_answer": user_answer,
        "correct_answer": correct_answer,
        "is_correct": is_correct,
        "interviewer_response": feedback
    }

def preload_next_audio(question_data, question_num):
    """Preload audio for next question while current one is being answered"""
    question_text = question_data.get('question', '')
    options = question_data.get('options', {})
    
    audio_text = f"Question {question_num}. {question_text}. Options: "
    
    for key, value in options.items():
        short_option = value[:50]  
        audio_text += f"{key}. {short_option}. "
    

    text_to_speech(audio_text)

def run_optimized_interview():
    user_message = input("👋 What topic would you like to be interviewed on? ")
    

    topic = user_message.split()[0] if user_message else "general knowledge"
    print(f"🎯 Topic: {topic}")
    
    text_to_speech(f"Starting interview on {topic}")
    
   
    print("🔄 Generating questions (optimized)...")
    all_questions = generate_questions_optimized(topic, num_questions=3)  
    
    if not all_questions:
        print("❌ Failed to generate questions. Please try again.")
        return
    
    score = 0
    total_questions = len(all_questions)
    
    print(f"\n📝 Starting interview with {total_questions} questions...")
    
    if total_questions > 0:
        preload_next_audio(all_questions[0], 1)
    
    for i, question in enumerate(all_questions, start=1):
        print(f"\n{'='*40}")
        print(f"🧠 Question {i}/{total_questions}:")
        print(f"Q: {question['question']}")
        
        for key, value in question['options'].items():
            print(f"   {key}) {value}")
        
        user_answer = input("\n👉 Your answer (A/B/C/D): ").strip().upper()
        
        result = evaluate_answer_optimized(question, user_answer)
        
        print(f"\n📊 Result:")
        print(f"   Your answer: {result['detected_answer']}")
        print(f"   Correct answer: {result['correct_answer']}")
        print(f"   🗣️ {result['interviewer_response']}")
        
        feedback_text = "Correct! " if result['is_correct'] else f"Incorrect. Correct was {result['correct_answer']}."
        text_to_speech(feedback_text)
        
        if result['is_correct']:
            score += 1
            print("   ✅ Correct!")
        else:
            print("   ❌ Incorrect")
        
        if i < total_questions:
            preload_next_audio(all_questions[i], i + 1)
    
    print(f"\n{'='*40}")
    print("🏁 Interview Finished!")
    print(f"📊 Final Score: {score}/{total_questions}")
    
    percentage = (score / total_questions) * 100
    final_text = f"You scored {score} out of {total_questions}."
    
    if percentage >= 80:
        final_text += " Excellent work!"
    elif percentage >= 60:
        final_text += " Good job!"
    else:
        final_text += " Keep practicing!"
    
    text_to_speech(final_text)
    print(f"🎯 {final_text}")



pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
run_optimized_interview()

🎯 Topic: Convolutional
🔄 Generating questions (optimized)...

📝 Starting interview with 3 questions...

🧠 Question 1/3:
Q: What is the primary goal of a convolutional layer?
   A) To perform element-wise multiplication
   B) To extract features from input data
   C) To apply pooling to reduce spatial dimensions
   D) To apply normalization to input values
⚠️ Audio error: Failed to connect. Probable cause: Unknown

📊 Result:
   Your answer: B
   Correct answer: B
   🗣️ Correct! Well done.
   ✅ Correct!

🧠 Question 2/3:
Q: What is the main advantage of using convolutional neural networks (CNNs) over traditional neural networks?
   A) Faster training times
   B) Better performance on image classification tasks
   C) Increased number of hidden layers
   D) Improved handling of sequential data

📊 Result:
   Your answer: C
   Correct answer: B
   🗣️ Incorrect. The right answer was B: Better performance on image classification tasks
   ❌ Incorrect

🧠 Question 3/3:
Q: What is the purpose of th

In [ ]:
from gtts import gTTS